In [1]:
!pip install unsloth trl peft accelerate bitsandbytes datasets mistral_common huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files
uploaded = files.upload()

# Step 3: Load and format dataset
import json
from datasets import Dataset

# Replace with the actual uploaded file name if different
dataset_path = "train.json"

# Load JSON data
with open(dataset_path, "r") as f:
    data = json.load(f)

# Format prompt for instruction tuning (user prompt → Python code output)
def format_prompt(example):
    return (
        f"<|begin_of_text|>\n"
        f"### Instruction:\n{example['prompt']}\n\n"
        f"### Response:\n{example['completion']}\n"
        f"<|end_of_text|>"
    )

# Apply formatting
formatted_data = [format_prompt(item) for item in data]

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": formatted_data})

# Optional: Split for training/validation
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Step 4: Inspect a few samples
print(dataset["train"][0]["text"])

Saving train.json to train.json
<|begin_of_text|>
### Instruction:
Create a hex nut of size M16-2 type iso4033

### Response:
from cq_warehouse.fastener import HexNut
result = HexNut(size="M16-2", fastener_type="iso4033", hand="right")
<|end_of_text|>


In [3]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [4]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-3B-Instruct"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.10.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"], # Select the 'train' split
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/412 [00:00<?, ? examples/s]

In [7]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 412 | Num Epochs = 3 | Total steps = 156
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Step,Training Loss
25,1.023200
50,0.143300
75,0.124500
100,0.114900
125,0.108000
150,0.105700


In [8]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt (CAD-style example)
messages = [
    {
        "role": "user",
        "content": "Create a countersunk screw of size M5-0.8 length 12mm type iso14582",
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Oct 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Create a countersunk screw of size M5-0.8 length 12mm type iso14582<|eot_id|><|start_header_id|>assistant<|end_header_id|>

From cq_warehouse.screw import CountersunkScrew
result = CountersunkScrew(size="M5-0.8", fastener_type="iso14582", hand="right", length="12mm")
<|end_of_text|>


In [9]:
!rm -rf llama.cpp
!git clone --recursive https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!cmake -B build
!cmake --build build --config Release
%cd ..

Cloning into 'llama.cpp'...
remote: Enumerating objects: 64734, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 64734 (delta 85), reused 35 (delta 35), pack-reused 64593 (from 3)
Receiving objects: 100% (64734/64734), 174.54 MiB | 30.83 MiB/s, done.
Resolving deltas: 100% (47023/47023), done.
/content/llama.cpp
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification

In [10]:
!mv /content/llama.cpp/build/bin/llama-quantize /content/llama.cpp/

In [11]:
# Save the fine-tuned model in a standard Hugging Face format
model.save_pretrained("hf_model")
tokenizer.save_pretrained("hf_model")

('hf_model/tokenizer_config.json',
 'hf_model/special_tokens_map.json',
 'hf_model/chat_template.jinja',
 'hf_model/tokenizer.json')

In [12]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base_model_name = "unsloth/Llama-3.2-3B-Instruct"  # ✅ full-precision version
adapter_path = "/content/hf_model"
merged_path = "/content/merged_model"

# Load base model (full precision)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,  # ✅ ensure FP16
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the LoRA adapter
model = PeftModel.from_pretrained(model, adapter_path)

# Merge LoRA weights
model = model.merge_and_unload()

# Ensure all tensors are FP16
model = model.to(torch.float16)

# Save merged model
model.save_pretrained(merged_path, safe_serialization=True)
tokenizer.save_pretrained(merged_path)

print("✅ Merged model saved successfully at:", merged_path)


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Merged model saved successfully at: /content/merged_model


In [13]:
!python3 /content/llama.cpp/convert_hf_to_gguf.py /content/merged_model --outfile /content/ask-cad.gguf

INFO:hf-to-gguf:Loading model: merged_model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.attn_k.weight

In [14]:
!/content/llama.cpp/llama-quantize /content/ask-cad.gguf /content/merged_model/unsloth.Q4_K_M.gguf q4_k_m

main: build = 6746 (f9bc66c3)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/ask-cad.gguf' to '/content/unsloth.Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 29 key-value pairs and 255 tensors from /content/ask-cad.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Merged_Model
llama_model_loader: - kv   3:                         general.size_label str              = 3.2B
llama_model_loader: - kv   4:                          llama.block_count u32              = 28
llama_model_loader: - kv   5:                       llama.context_length u32      

In [17]:
# Enter Hugging Face token and repo name (created if needed)
from getpass import getpass
hf_token = getpass('Enter your Hugging Face token: ')
repo_name = input('Enter target HF repo name (e.g. username/deepseek-coder-6.7b-lora): ').strip()
print('Repo will be:', repo_name)


Enter your Hugging Face token: ··········
Enter target HF repo name (e.g. username/deepseek-coder-6.7b-lora): Pranu999/ask-cad
Repo will be: Pranu999/ask-cad


In [21]:
# Push adapter to Hugging Face Hub
from huggingface_hub import HfApi
import os

assert 'hf_token' in globals() and 'repo_name' in globals(), 'hf_token and repo_name must be set'

api = HfApi()
print('Creating repo if not exists...')
api.create_repo(repo_id=repo_name, token=hf_token, exist_ok=True)

folder = '/content/merged_model'
print('Uploading folder', folder, 'to', repo_name)
api.upload_folder(folder_path=folder, path_in_repo='', repo_id=repo_name, token=hf_token)
print('Upload complete!')

Creating repo if not exists...
Uploading file /content/unsloth.Q4_K_M.gguf to Pranu999/ask-cad


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  /content/unsloth.Q4_K_M.gguf:   1%|          | 16.7MB / 2.02GB            

Upload complete!


In [ ]:
from google.colab import files
files.download("/content/merged_model/unsloth.Q4_K_M.gguf")